In [ ]:
#p1
import math
import random

def strength(x):
    return math.log2(x + 1) + x / 10

def utility(maxV, minV):
    i = random.randint(0, 1)
    rand_val = random.randint(1, 10)
    return strength(maxV) - strength(minV) + ((-1) ** i) * rand_val / 10

def minimax(depth, node_index, is_maximizing, values, alpha, beta):
    if depth == 0:
        return values[node_index]

    if is_maximizing:
        best = float('-inf')
        for i in range(2):
            val = minimax(depth - 1, node_index * 2 + i, False, values, alpha, beta)
            best = max(best, val)
            alpha = max(alpha, best)
            if beta <= alpha:
                break
        return best
    else:
        best = float('inf')
        for i in range(2):
            val = minimax(depth - 1, node_index * 2 + i, True, values, alpha, beta)
            best = min(best, val)
            beta = min(beta, best)
            if beta <= alpha:
                break
        return best

def play_game(starting_player, carlsen_strength, caruana_strength):
    max_player_name = "Magnus Carlsen" if starting_player == 0 else "Fabiano Caruana"
    min_player_name = "Fabiano Caruana" if starting_player == 0 else "Magnus Carlsen"

    maxV = carlsen_strength if starting_player == 0 else caruana_strength
    minV = caruana_strength if starting_player == 0 else carlsen_strength


    leaves = [utility(maxV, minV) for _ in range(32)]
    result = minimax(5, 0, True, leaves, float('-inf'), float('inf'))

    if result > 0:
        winner = f"{max_player_name} (Max)"
    elif result < 0:
        winner = f"{min_player_name} (Min)"
    else:
        winner = "Draw"

    return result, winner

def main():
    starting_player = int(input("Enter starting player for game 1 (0 for Carlsen, 1 for Caruana): "))
    carlsen_strength = float(input("Enter base strength for Carlsen: "))
    caruana_strength = float(input("Enter base strength for Caruana: "))

    results = []
    carlsen_wins = 0
    caruana_wins = 0
    draws = 0

    for game in range(4):
        current_start = (starting_player + game) % 2
        result, winner = play_game(current_start, carlsen_strength, caruana_strength)
        results.append((game + 1, result, winner))

        print(f"Game {game + 1} Winner: {winner} (Utility value: {round(result, 2)})")

        if "Carlsen" in winner:
            if "Draw" not in winner:
                carlsen_wins += 1
        elif "Caruana" in winner:
            caruana_wins += 1
        else:
            draws += 1

    print("\nOverall Results:")
    print(f"Magnus Carlsen Wins: {carlsen_wins}")
    print(f"Fabiano Caruana Wins: {caruana_wins}")
    print(f"Draws: {4 - carlsen_wins - caruana_wins}")

    if carlsen_wins > caruana_wins:
        print("Overall Winner: Magnus Carlsen")
    elif caruana_wins > carlsen_wins:
        print("Overall Winner: Fabiano Caruana")
    else:
        print("Overall Winner: Draw")

if __name__ == "__main__":
    main()




#p2

import random
import math
import sys


sys.setrecursionlimit(2000)


def compute_power(level):
    return math.log2(level + 1) + level / 10


def create_outcomes(p1, p2):
    outcomes = []
    final_nodes = 2 ** 5
    for _ in range(final_nodes):
        bonus = random.uniform(-1, 1)
        diff = compute_power(p1) - compute_power(p2) + bonus
        outcomes.append(round(diff, 2))
    return outcomes


def standard_minimax(tree, depth, pos, a, b, is_max):
    if depth == 5:
        return tree[pos]

    if is_max:
        best = -float('inf')
        for i in range(2):
            val = standard_minimax(tree, depth + 1, pos * 2 + i, a, b, False)
            best = max(best, val)
            a = max(a, best)
            if b <= a:
                break
        return best
    else:
        best = float('inf')
        for i in range(2):
            val = standard_minimax(tree, depth + 1, pos * 2 + i, a, b, True)
            best = min(best, val)
            b = min(b, best)
            if b <= a:
                break
        return best


def mind_control_mode(tree, depth, pos):


    if depth >= 5:
        return tree[pos]

    best = -float('inf')
    for i in range(2):
        for j in range(2):
            child_index = ((pos * 2 + i) * 2 + j)
            if child_index >= len(tree):
                continue
            val = mind_control_mode(tree, depth + 2, child_index)
            best = max(best, val)
    return best


def start_simulation():
    print("Who moves first? (0 = Light, 1 = L)")
    first = int(input("Your input: "))
    penalty = float(input("Mind Control Cost: "))
    light_lvl = float(input("Light's Strength: "))
    l_lvl = float(input("L's Strength: "))

    max_player = light_lvl if first == 0 else l_lvl
    min_player = l_lvl if first == 0 else light_lvl

    game_leafs = create_outcomes(max_player, min_player)

    normal_score = standard_minimax(game_leafs, 0, 0, -float('inf'), float('inf'), True)
    magic_score = mind_control_mode(game_leafs, 0, 0)
    net_magic_score = magic_score - penalty

    print(f"\n>> Standard Strategy Value: {round(normal_score, 2)}")
    print(f">> Mind Control Value: {round(magic_score, 2)}")
    print(f">> After Mind Control Cost: {round(net_magic_score, 2)}")

    if net_magic_score > normal_score:
        print("\n** Recommendation: Use Mind Control. **")
    else:
        print("\n** Recommendation: Avoid Mind Control. **")


start_simulation()